In [15]:
import numpy as np
import time
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, LSTM
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam
import os

In [16]:
Y = np.loadtxt("datasets/PruebaY2.0.csv", delimiter=",")
X = np.loadtxt("datasets/PruebaX2.0.csv", delimiter=",")

In [17]:
print(X.shape)
print(Y.shape)
print(np.amax(X))
print(np.amin(X))
print(np.amax(Y))
print(np.amin(Y))
print(Y[0:,4:5].shape)

(497, 1100)
(497, 5)
127.0
-128.0
6.0
0.0
(497, 1)


In [80]:
#X_formated = tf.keras.utils.normalize(X, axis=1)
X_math = (((X - np.amin(X)) * (1)) / (np.amax(X) - np.amin(X)))
X_math = np.round(X_math, decimals=1)
#X_formated = X_math.reshape((497,11,100))
X_formated = (X / 128).reshape((497,100,11))

In [81]:
print(np.amax(X_formated))
print(np.amin(X_formated))

0.9921875
-1.0


In [82]:
Y_formated = Y.astype(int)
Y_formated[0:,1]

array([0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0,
       1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1,
       2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2,
       3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3,
       4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4,
       5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5,
       6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6,
       0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0,
       1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1,
       2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2,
       3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3,
       4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4,
       5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5,
       6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6,

In [83]:
#config = tf.ConfigProto( device_count = {'GPU': 1})
#sess = tf.Session(config=config)
#tf.keras.backend.set_session(sess)

In [87]:
model = Sequential()

model.add(LSTM(128, input_shape=(X_formated.shape[1:]), activation='relu', return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(128, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(7, activation='softmax'))


In [88]:
#tensorboard = TensorBoard(log_dir=NAME)
#adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.01, amsgrad=False)
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [89]:
model.fit(X_formated, Y_formated[0:,1], epochs=5, validation_split=0.10)

Train on 447 samples, validate on 50 samples
Epoch 1/5
447/447 [==============================] - 6s 14ms/step - loss: 1.9465 - acc: 0.1342 - val_loss: 1.9456 - val_acc: 0.1400
Epoch 2/5
447/447 [==============================] - 1s 2ms/step - loss: 1.9457 - acc: 0.1342 - val_loss: 1.9431 - val_acc: 0.1200
Epoch 3/5
447/447 [==============================] - 1s 3ms/step - loss: 1.9419 - acc: 0.1566 - val_loss: 1.9363 - val_acc: 0.1200
Epoch 4/5
447/447 [==============================] - 1s 2ms/step - loss: 1.9394 - acc: 0.1454 - val_loss: 1.9131 - val_acc: 0.1600
Epoch 5/5
447/447 [==============================] - 1s 2ms/step - loss: 1.9354 - acc: 0.1298 - val_loss: 1.8926 - val_acc: 0.2000


In [37]:
predictions = model.predict(X_formated)

In [99]:
np.argmax(predictions[1])
for x in predictions:
    print(x)

[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0

In [90]:
val_loss, val_acc = model.evaluate(X_formated, Y_formated[0:,1])
print(val_loss)
print(val_acc)

497/497 [==============================] - 0s 489us/step
1.8974623824029382
0.18712273644849328


In [88]:
model.save("modelo.h5")